In [ ]:
import numpy as np

def random_tt_binary_fixed(seed=None):
    """
    Create a random TT with 7 binary modes and ranks [1,2,4,8,8,4,2,1].

    Parameters
    ----------
    seed : int or None
        Random seed for reproducibility.

    Returns
    -------
    cores : list of ndarray
        List of 7 cores, shapes as described.
    """
    if seed is not None:
        np.random.seed(seed)
    dims  = [2] * 7
    ranks = [1, 2, 4, 8, 8, 4, 2, 1]
    cores = []
    for k in range(7):
        rL, n, rR = ranks[k], dims[k], ranks[k+1]
        G = np.random.randn(rL, n, rR)
        cores.append(G)
    return cores

def tt_to_tensor(cores):
    """
    Reconstruct the full tensor from its TT cores.
    """
    # start with first core, squeeze r0=1
    A = np.squeeze(cores[0], axis=0)   # shape (2, r1)
    for G in cores[1:]:
        A = np.tensordot(A, G, axes=([-1],[0]))
    # squeeze final r7=1
    return np.squeeze(A, axis=-1)      # final shape (2,2,...,2), 7 modes

if __name__ == "__main__":
    cores = random_tt_binary_fixed(seed=2025)
    print("Core shapes:", [c.shape for c in cores])
    A = tt_to_tensor(cores)
    print("Reconstructed tensor shape:", A.shape)
    # Now A[i1,i2,...,i7] exists for each ik∈{0,1}

print(type(cores))
print(type(cores[0]))

Core shapes: [(1, 2, 2), (2, 2, 4), (4, 2, 8), (8, 2, 8), (8, 2, 4), (4, 2, 2), (2, 2, 1)]
Reconstructed tensor shape: (2, 2, 2, 2, 2, 2, 2)
[[[[[[[ 1.55659199e+01 -1.25428262e+01]
      [-3.80965334e+01  1.86933722e+01]]

     [[ 1.13796505e+01 -8.49975516e+00]
      [ 1.93176048e+01 -9.66352483e+00]]]


    [[[ 6.78052320e+00 -5.07435639e+00]
      [ 1.76490808e+01 -9.85224887e+00]]

     [[-1.80032178e-01 -6.14789809e-01]
      [-1.95599646e+01  1.01208276e+01]]]]



   [[[[ 8.79991879e+00 -6.66049887e+00]
      [-4.73120106e+00  8.22042644e-01]]

     [[ 9.44189471e-02 -6.80003102e-01]
      [ 2.90251538e+01 -1.49479052e+01]]]


    [[[ 3.38758957e+00 -2.96544438e+00]
      [ 2.93388712e+01 -1.66146605e+01]]

     [[-5.71554671e+00  3.10978717e+00]
      [-2.37963114e+01  1.20484610e+01]]]]]




  [[[[[-5.78919570e-01  5.61183772e-01]
      [-4.22885871e+00  2.16094787e+00]]

     [[-1.02675943e+00  6.05000791e-01]
      [ 7.71919813e+00 -4.07361487e+00]]]


    [[[-3.40431807e-01 

In [2]:
W = []
nk = np.array([1,2,4,8,8,4,2])

for k in range(len(cores)):
        
        j, i, n_k = cores[k].shape
        #print(cores[k].shape)
        cores[k] = np.reshape(cores[k], (j * i, n_k)) 
        #print(cores[k].shape)       
        U, S, V = np.linalg.svd(cores[k])
        R = np.diag(S) @ V
        if k != len(cores)-1:
            cores[k+1] = np.einsum('j r,r i k->j i k', R, cores[k+1])
        W.append(U)


for k, U in enumerate(W):
    print(U.shape)
    start = k + 1
    mn    = min(start, np.log2(nk[k])) 
    diff  = int(start - mn)
    print(f"This unitary acts from qubit {int(diff)} to qubit {start}")

    #for i in range(0,diff+1):
    #     U = np.kron(np.eye(2),U)

    #for j in range(start+1,len(W)):
    #     U = np.kron(U,np.eye(2))

    print(U.shape)


(2, 2)
This unitary acts from qubit 1 to qubit 1
(2, 2)
(4, 4)
This unitary acts from qubit 1 to qubit 2
(4, 4)
(8, 8)
This unitary acts from qubit 1 to qubit 3
(8, 8)
(16, 16)
This unitary acts from qubit 1 to qubit 4
(16, 16)
(16, 16)
This unitary acts from qubit 2 to qubit 5
(16, 16)
(8, 8)
This unitary acts from qubit 4 to qubit 6
(8, 8)
(4, 4)
This unitary acts from qubit 6 to qubit 7
(4, 4)
